In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
from ipywidgets import FloatProgress
from time import sleep
from IPython.display import display
import re
import pickle
from urlextract import URLExtract
import json
import sqlite3

In [2]:
result=requests.get(f'https://www.shopclues.com/mens-clothing.html?page=1')
get= result.content
soup = BeautifulSoup(get,"html.parser")

In [3]:
s=soup.find_all('div',class_='column col4')

In [4]:
s[0].find_all('a')[1].attrs['href']

'//www.shopclues.com/stylogue-round-neck-half-sleeve-t-shirt-for-men-150492191.html'

In [5]:
product_link=[]
c=0
while (c <400):
    c=c+1
    result=requests.get(f'https://www.shopclues.com/mens-clothing.html?page={c}')
    g= result.content
    soup = BeautifulSoup(g,"html.parser")
    data=soup.find_all('div',class_='column col4')
    for i in data:
        product_link.append("https:"+i.find_all('a')[1].attrs['href'])

In [6]:
len(product_link)

9600

In [7]:
s=list(set(product_link))

In [8]:
len(s)

2401

In [71]:
result=requests.get(f'https://www.shopclues.com/freed-minds-dotted-white-casual-shirt-for-men-141084581.html')
get= result.content
soup = BeautifulSoup(get,"html.parser")

In [72]:
s=soup.find('div',class_='prd_mid_info')

In [73]:
#s

In [74]:
s.find('link',{'itemprop':'availability'}).attrs['href']
#<link href="https://schema.org/InStock" itemprop="availability"/>

'https://schema.org/InStock'

In [75]:
img=soup.find('div',class_='prd_img_gallery')
img_list=img.find_all('li',class_="thumb_click")

In [76]:
img_list[0].a.attrs['data-zoom-image']

'https://cdn.shopclues.com/images1/detailed/92880/141084581-92880242-1537502857.jpg'

In [10]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
#Gender
Description=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [11]:
error_link=[]
out_of_stock=[]
for i in s:
    try:
        result = requests.get(i)
        c = result.content
        soup = BeautifulSoup(c,"html.parser")
        s=soup.find('div',class_='prd_mid_info')
        
        if(list(s.find('link',{'itemprop':'availability'}).attrs.values())[1]=='https://schema.org/InStock'):
            
            if(s.find('span',{'id':"sec_list_price_"})!=None):
                MRP.append(s.find('span',{'id':"sec_list_price_"}).text.split('MRP:')[1])
            else:
                if(s.find('span',{'id':'o_price'})!=None):
                    MRP.append(s.find('span',{'id':"o_price"}).text)
                else:
                    MRP.append(s.find('span',{'id':"f_price"}).text)
            Product_Link.append(i)
            Product_Name.append(s.find('h1',{'itemprop':'name'}).text.rstrip().lstrip())
            Product_Brand.append(s.find('span',{'itemprop':'brand'}).text)
            size=s.find('div',{'class':"f_variation"})
            if(size==None):
                Size_Available.append("Not Available")
            else:
                li=size.ul.find_all('span',{'class':'variant'})
                s_l=''
                for j in li:
                    s_l=s_l+j.text+" , "
                Size_Available.append(s_l[:-2])
            Price.append(s.find('span',{'class':'f_price'}).text)
           
                
            
            Description.append(s.find('span',{'itemprop':'description'}).text)
            img=soup.find('div',class_='prd_img_gallery')
            img_list=img.find_all('li',class_="thumb_click")
            if(len(img_list)==1):
                Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                Secondary_Image_Link.append("Not Available")
            else:
                Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                Secondary_Image_Link.append(img_list[1].a.attrs['data-zoom-image'])
        else:
            out_of_stock.append(i)
            
    except:
        error_link.append(i)
        pass

In [17]:
data=pd.DataFrame({
    "Product":Product_Name,
    "Link":Product_Link,
    "Brand":Product_Brand,
    "Size Available":Size_Available,
    "Selling Price":Price,
    "MRP":MRP,
    "Gender":'Men',
    "Description":Description,
    "Primary_Image":Primary_Image_Link,
    "Secondary_Image":Secondary_Image_Link,
    
})

In [18]:
data.to_csv('Men_Clothing_SC.csv',index=False)

In [19]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
#Gender
Description=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [20]:
#out_of_stock=[]
err=[]
for i in error_link:
    try:
        result = requests.get(i)
        c = result.content
        soup = BeautifulSoup(c,"html.parser")
        s=soup.find('div',class_='prd_mid_info')
        
        if(list(s.find('link',{'itemprop':'availability'}).attrs.values())[1]=='https://schema.org/InStock'):
            
            
            if(s.find('span',{'class':'o_price'})!=None):
                MRP.append(s.find('span',{'class':"o_price"}).text)
            else:
                MRP.append(s.find('span',{'class':"f_price"}).text)
            Product_Link.append(i)
            Product_Name.append(s.find('h1',{'itemprop':'name'}).text.rstrip().lstrip())
            Product_Brand.append(s.find('span',{'itemprop':'brand'}).text)
            size=s.find('div',{'class':"f_variation"})
            if(size==None):
                Size_Available.append("Not Available")
            else:
                li=size.ul.find_all('span',{'class':'variant'})
                s_l=''
                for j in li:
                    s_l=s_l+j.text+" , "
                Size_Available.append(s_l[:-2])
            Price.append(s.find('span',{'class':'f_price'}).text)
           
                
            
            Description.append(s.find('span',{'itemprop':'description'}).text)
            img=soup.find('div',class_='prd_img_gallery')
            img_list=img.find_all('li',class_="thumb_click")
            if(len(img_list)==1):
                Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                Secondary_Image_Link.append("Not Available")
            else:
                Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                Secondary_Image_Link.append(img_list[1].a.attrs['data-zoom-image'])
        else:
            out_of_stock.append(i)
            
    except:
        err.append(i)
        pass

In [22]:
data=pd.DataFrame({
    "Product":Product_Name,
    "Link":Product_Link,
    "Brand":Product_Brand,
    "Size Available":Size_Available,
    "Selling Price":Price,
    "MRP":MRP,
    "Gender":'Men',
    "Description":Description,
    "Primary_Image":Primary_Image_Link,
    "Secondary_Image":Secondary_Image_Link,
    
})

In [23]:
new=pd.read_csv('Men_Clothing_SC.csv')

In [24]:
n=new.append(data)

In [25]:
n.to_csv('Men_Clothing_Sc.csv')

In [26]:
n.shape

(2399, 10)

In [27]:
out_of_stock

['https://www.shopclues.com/dia-a-dia-unisex-combo-of-3-cotton-shorts-143432323.html',
 'https://www.shopclues.com/ketex-pack-of-navy-blue-black-bottle-green-polo-t-shirts-pack-of-3-143392593.html']

In [29]:
acc_link=[]
products=['footwear-mens-shoes','men-watches','accessories-men-belts','mens-accessories-wallets']
for i in products:
    c = 0
    while (c <400):
        c=c+1
        result=requests.get(f'https://www.shopclues.com/{i}.html?page={c}')
        g= result.content
        soup = BeautifulSoup(g,"html.parser")
        data=soup.find_all('div',class_='column col4')
        for j in data:
            acc_link.append("https:"+j.find_all('a')[1].attrs['href'])

In [31]:
men_acc=list(set(acc_link))

In [32]:
len(men_acc)

7676

In [33]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
#Gender
Description=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [34]:
error_link=[]
out_of_stock=[]
for i in men_acc:
    try:
        result = requests.get(i)
        c = result.content
        soup = BeautifulSoup(c,"html.parser")
        s=soup.find('div',class_='prd_mid_info')
        
        if(list(s.find('link',{'itemprop':'availability'}).attrs.values())[1]=='https://schema.org/InStock'):
            
            if(s.find('span',{'id':"sec_list_price_"})!=None):
                MRP.append(s.find('span',{'id':"sec_list_price_"}).text.split('MRP:')[1])
            else:
                if(s.find('span',{'id':'o_price'})!=None):
                    MRP.append(s.find('span',{'id':"o_price"}).text)
                else:
                    MRP.append(s.find('span',{'id':"f_price"}).text)
            Product_Link.append(i)
            Product_Name.append(s.find('h1',{'itemprop':'name'}).text.rstrip().lstrip())
            Product_Brand.append(s.find('span',{'itemprop':'brand'}).text)
            size=s.find('div',{'class':"f_variation"})
            if(size==None):
                Size_Available.append("Not Available")
            else:
                li=size.ul.find_all('span',{'class':'variant'})
                s_l=''
                for j in li:
                    s_l=s_l+j.text+" , "
                Size_Available.append(s_l[:-2])
            Price.append(s.find('span',{'class':'f_price'}).text)
           
                
            
            Description.append(s.find('span',{'itemprop':'description'}).text)
            img=soup.find('div',class_='prd_img_gallery')
            img_list=img.find_all('li',class_="thumb_click")
            if(len(img_list)==1):
                Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                Secondary_Image_Link.append("Not Available")
            else:
                Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                Secondary_Image_Link.append(img_list[1].a.attrs['data-zoom-image'])
        else:
            out_of_stock.append(i)
            
    except:
        error_link.append(i)
        pass

In [35]:
len(error_link)

127

In [36]:
len(out_of_stock)

3

In [37]:
data=pd.DataFrame({
    "Product":Product_Name,
    "Link":Product_Link,
    "Brand":Product_Brand,
    "Size Available":Size_Available,
    "Selling Price":Price,
    "MRP":MRP,
    "Gender":'Men',
    "Description":Description,
    "Primary_Image":Primary_Image_Link,
    "Secondary_Image":Secondary_Image_Link,
    
})

In [38]:
#data.to_csv('Men_all_SC.csv')

In [39]:
data.shape

(7546, 10)

In [40]:
#out_of_stock=[]
err=[]
for i in error_link:
    try:
        result = requests.get(i)
        c = result.content
        soup = BeautifulSoup(c,"html.parser")
        s=soup.find('div',class_='prd_mid_info')
        
        if(list(s.find('link',{'itemprop':'availability'}).attrs.values())[1]=='https://schema.org/InStock'):
            
            
            if(s.find('span',{'class':'o_price'})!=None):
                MRP.append(s.find('span',{'class':"o_price"}).text)
            else:
                MRP.append(s.find('span',{'class':"f_price"}).text)
            Product_Link.append(i)
            Product_Name.append(s.find('h1',{'itemprop':'name'}).text.rstrip().lstrip())
            Product_Brand.append(s.find('span',{'itemprop':'brand'}).text)
            size=s.find('div',{'class':"f_variation"})
            if(size==None):
                Size_Available.append("Not Available")
            else:
                li=size.ul.find_all('span',{'class':'variant'})
                s_l=''
                for j in li:
                    s_l=s_l+j.text+" , "
                Size_Available.append(s_l[:-2])
            Price.append(s.find('span',{'class':'f_price'}).text)
           
                
            
            Description.append(s.find('span',{'itemprop':'description'}).text)
            img=soup.find('div',class_='prd_img_gallery')
            img_list=img.find_all('li',class_="thumb_click")
            if(len(img_list)==1):
                Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                Secondary_Image_Link.append("Not Available")
            else:
                Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                Secondary_Image_Link.append(img_list[1].a.attrs['data-zoom-image'])
        else:
            out_of_stock.append(i)
            
    except:
        err.append(i)
        pass

In [50]:
data=pd.DataFrame({
    "Product":Product_Name,
    "Link":Product_Link,
    "Brand":Product_Brand,
    "Size Available":Size_Available,
    "Selling Price":Price,
    "MRP":MRP,
    "Gender":'Men',
    "Description":Description,
    "Primary_Image":Primary_Image_Link,
    "Secondary_Image":Secondary_Image_Link,
    
})

In [51]:
data.shape

(7673, 10)

In [52]:
data.to_csv('Men_all_SC.csv',index=False)

In [57]:
n=pd.read_csv('Men_Clothing_SC.csv')

In [59]:
n.columns

Index(['Unnamed: 0', 'Product', 'Link', 'Brand', 'Size Available',
       'Selling Price', 'MRP', 'Gender', 'Description', 'Primary_Image',
       'Secondary_Image'],
      dtype='object')

In [61]:
new=n.drop(columns='Unnamed: 0',axis=1)

In [64]:
data=data.append(new)

In [65]:
data.shape

(10072, 10)

In [66]:
data.to_csv('All_Men_SC.csv',index=False)